In [ ]:
import json
import os
from dotenv import load_dotenv
from openai import AzureOpenAI
import PyPDF2

# Load environment variables from .env file
load_dotenv()

azure_api_key = os.getenv('AZURE_API_KEY')
azure_endpoint = os.getenv('AZURE_ENDPOINT')
azure_api_version = os.getenv('AZURE_API_VERSION')
model_name = os.getenv('MODEL_NAME')

print(f"Azure API Key: {azure_api_key}, Azure Endpoint: {azure_endpoint}, Azure API Version: {azure_api_version}, Model name: {model_name}")

In [ ]:
client = AzureOpenAI(
    api_key=azure_api_key,
    api_version=azure_api_version,
    azure_endpoint=azure_endpoint
)

In [ ]:
# Function to extract text from a PDF file
def extract_text_from_pdf(pdf_file):
    pdf_reader = PyPDF2.PdfReader(pdf_file)
    text = ''
    for page_num in range(len(pdf_reader.pages)):
        page = pdf_reader.pages[page_num]
        text += page.extract_text()
    return text

In [ ]:
# Example usage:
pdf_file_path = 'example.pdf'  # Replace with your PDF file path
pdf_text = extract_text_from_pdf(pdf_file_path)
print("Extracted PDF Text:")
print(pdf_text)

In [ ]:
system_prompt = """You are a personal assistant to a developer. Your role is to answer user questions politely and competently.
    You should follow these instructions to solve the case:
    - You are provided with a PDF text to extract information.
    - Understand the ask and get relevant instructions.
    - Follow the instructions to provide solutions to users.
    - Never modify the user query.
    - Never modify the answer generated from the tool. However, make the content more user-friendly by formatting it in markdown, including headers, bullet points, and other markdown elements as appropriate.

    Only call a tool once in a single message.
    If you don't find an answer from the provided PDF document text, give a clear, say 'I cannot answer.' Never provide any dummy values.
"""

In [ ]:
messages = [
        {
            "role": "system",
            "content": system_prompt
        },
        {
            "role": "user",
            "content": f'''PDF text: {pdf_text}'''
        }
    ]

In [ ]:
# Example integration with OpenAI
print(f"Azure API Key: {azure_api_key}, Azure Endpoint: {azure_endpoint}, Azure API Version: {azure_api_version}, Model name: {model_name}")

def ask_question(messages, user_query):
    # Append the new user query to the messages list
    messages.append({
        "role": "user",
        "content": user_query
    })

    try:
        response = client.chat.completions.create(
            model=model_name,
            messages=messages,
            temperature=0,
            max_tokens=500,
            frequency_penalty=0,
            presence_penalty=0,
            stop=None,
        )
        
        final_result = json.loads(response.model_dump_json(indent=2))
        model_reply = final_result['choices'][0]['message']['content']
        
        # Append the AI's response to the messages list
        messages.append({
            "role": "assistant",
            "content": model_reply
        })
        
        return model_reply
    except json.JSONDecodeError as e:
        return f"JSON decode error: {e}"
    except Exception as e:
        return f"Unexpected error: {e}"


In [ ]:
# Interactive Loop
while True:
    user_query = input("Enter your question (or 'exit' to quit): ")
    if user_query.lower() == 'exit':
        break
    answer = ask_question(messages, user_query)
    print(f"AI Answer: {answer}")